In [12]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.optim import lr_scheduler

import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.7.0
Torchvision Version:  0.8.1


In [18]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.RandomErasing()
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/home/hui/data/kaggle/jincetest/indoor_scene'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Number of classes in the dataset
num_classes = 67

# Batch size for training (change depending on how much memory you have)
batch_size = 32

# Number of epochs to train for 
num_epochs = 60

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract = True

In [19]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=20):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):            
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):

                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [20]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [27]:
model_ft = models.resnet18(pretrained=True)
set_parameter_requires_grad(model_ft, feature_extract)
num_ftrs = model_ft.fc.in_features

# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 67)

model_ft = model_ft.to(device)


criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 20 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [28]:
model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/59
----------
train Loss: 3.7591 Acc: 0.1372
val Loss: 2.8551 Acc: 0.3577

Epoch 1/59
----------
train Loss: 2.6568 Acc: 0.3941
val Loss: 1.9514 Acc: 0.5288

Epoch 2/59
----------
train Loss: 2.0825 Acc: 0.4969
val Loss: 1.5679 Acc: 0.5915

Epoch 3/59
----------
train Loss: 1.7889 Acc: 0.5510
val Loss: 1.3948 Acc: 0.6341

Epoch 4/59
----------
train Loss: 1.5966 Acc: 0.5968
val Loss: 1.2601 Acc: 0.6639

Epoch 5/59
----------
train Loss: 1.4521 Acc: 0.6251
val Loss: 1.1879 Acc: 0.6796

Epoch 6/59
----------
train Loss: 1.3585 Acc: 0.6410
val Loss: 1.1251 Acc: 0.6990

Epoch 7/59
----------
train Loss: 1.2724 Acc: 0.6604
val Loss: 1.0809 Acc: 0.6960

Epoch 8/59
----------
train Loss: 1.2068 Acc: 0.6789
val Loss: 1.0236 Acc: 0.7170

Epoch 9/59
----------
train Loss: 1.1380 Acc: 0.6936
val Loss: 1.0393 Acc: 0.7199

Epoch 10/59
----------
train Loss: 1.0810 Acc: 0.7089
val Loss: 1.0213 Acc: 0.7065

Epoch 11/59
----------
train Loss: 1.0155 Acc: 0.7279
val Loss: 1.0082 Acc: 0.7117

Ep

In [29]:
feature_extract = False

In [30]:
feature_extract = False
set_parameter_requires_grad(model_ft, feature_extract)


In [31]:
model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/59
----------
train Loss: 0.4200 Acc: 0.8901
val Loss: 1.0219 Acc: 0.7371

Epoch 1/59
----------
train Loss: 0.4104 Acc: 0.8929
val Loss: 1.0156 Acc: 0.7274

Epoch 2/59
----------
train Loss: 0.4210 Acc: 0.8856
val Loss: 1.0021 Acc: 0.7296

Epoch 3/59
----------
train Loss: 0.4092 Acc: 0.8927
val Loss: 0.9913 Acc: 0.7349

Epoch 4/59
----------
train Loss: 0.3899 Acc: 0.8938
val Loss: 0.9746 Acc: 0.7349

Epoch 5/59
----------
train Loss: 0.4060 Acc: 0.8921
val Loss: 0.9686 Acc: 0.7334

Epoch 6/59
----------
train Loss: 0.3915 Acc: 0.8942
val Loss: 0.9690 Acc: 0.7386

Epoch 7/59
----------
train Loss: 0.3853 Acc: 0.8990
val Loss: 0.9794 Acc: 0.7386

Epoch 8/59
----------
train Loss: 0.3788 Acc: 0.8949
val Loss: 1.0061 Acc: 0.7379

Epoch 9/59
----------
train Loss: 0.3896 Acc: 0.8970
val Loss: 0.9870 Acc: 0.7401

Epoch 10/59
----------
train Loss: 0.3948 Acc: 0.8925
val Loss: 1.0290 Acc: 0.7237

Epoch 11/59
----------
train Loss: 0.3631 Acc: 0.9009
val Loss: 1.0288 Acc: 0.7304

Ep

In [33]:
model_ft = models.resnet50(pretrained=True)
set_parameter_requires_grad(model_ft, feature_extract)
num_ftrs = model_ft.fc.in_features

# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 67)

model_ft = model_ft.to(device)


criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 9 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=9, gamma=0.1)


In [34]:
model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/59
----------
train Loss: 3.7466 Acc: 0.1799
val Loss: 2.6502 Acc: 0.4511

Epoch 1/59
----------
train Loss: 2.4643 Acc: 0.4600
val Loss: 1.6434 Acc: 0.6042

Epoch 2/59
----------
train Loss: 1.7965 Acc: 0.5686
val Loss: 1.2607 Acc: 0.6677

Epoch 3/59
----------
train Loss: 1.4758 Acc: 0.6249
val Loss: 1.0946 Acc: 0.6945

Epoch 4/59
----------
train Loss: 1.2965 Acc: 0.6647
val Loss: 0.9963 Acc: 0.7184

Epoch 5/59
----------
train Loss: 1.1539 Acc: 0.6955
val Loss: 0.9239 Acc: 0.7319

Epoch 6/59
----------
train Loss: 1.0460 Acc: 0.7164
val Loss: 0.8757 Acc: 0.7431

Epoch 7/59
----------
train Loss: 0.9890 Acc: 0.7347
val Loss: 0.8534 Acc: 0.7647

Epoch 8/59
----------
train Loss: 0.9043 Acc: 0.7552
val Loss: 0.8141 Acc: 0.7677

Epoch 9/59
----------
train Loss: 0.8489 Acc: 0.7632
val Loss: 0.8298 Acc: 0.7730

Epoch 10/59
----------
train Loss: 0.7983 Acc: 0.7800
val Loss: 0.8207 Acc: 0.7610

Epoch 11/59
----------
train Loss: 0.7233 Acc: 0.8097
val Loss: 0.7884 Acc: 0.7647

Ep

In [36]:
torch.save(model_ft.state_dict(), './model/bst_resnet50.pt')